<a href="https://colab.research.google.com/github/Arshit162/Embedded-System/blob/main/Copy_of_Lab03_TensorFlow_vs_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 03: TensorFlow vs. PyTorch
- Train a model on MNIST in both TensorFlow and PyTorch, convert to TFLite and ONNX.  
- Use tf.GradientTape for Tensorflow custom training loop.



## TensorFlow Implementation

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Fill in normalization factor
x_test = x_test / 255.0     # Fill in normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),        # Fill input shape
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),  # Fill number of hidden neurons
    tf.keras.layers.Dense(10, activation='softmax')  # Fill number of output neurons
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',       # Fill name of loss function
              metrics=['accuracy'])

start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end - start} seconds")       # Output training time
model.evaluate(x_test, y_test)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 0.8608 - loss: 0.5018
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9550 - loss: 0.1563
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9660 - loss: 0.1133
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9732 - loss: 0.0889
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9784 - loss: 0.0731
TF Training time: 39.10148572921753 seconds
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9660 - loss: 0.1064


[0.09652259200811386, 0.9704999923706055]

## Convert TensorFlow model to TFLite

In [2]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpcgekeg3w'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  132947775974928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132947775979344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132947775978192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132947775977808: TensorSpec(shape=(), dtype=tf.resource, name=None)


## PyTorch Implementation

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
train_loader = DataLoader(datasets.MNIST(root='./data', train=True, transform=transform, download=True), batch_size=32)
test_loader = DataLoader(datasets.MNIST(root='./data', train=False, transform=transform, download=True), batch_size=1000)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 64)    # Fill correct input and output size
        self.fc2 = nn.Linear(64, 10)    # Fill correct input and output size
    def forward(self, x):
        x = F.relu(self.fc1(x))    # Fill correct layer
        return self.fc2(x)    # Fill correct layer

model = Net()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

start = time.time()
for epoch in range(5):
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()
print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")

100%|██████████| 9.91M/9.91M [00:00<00:00, 37.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.09MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 8.55MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.78MB/s]


PyTorch Training time: 61.99 seconds
Test accuracy: 0.9675


## Convert PyTorch model to ONNX

In [4]:
# Install ONNX
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 86.1 MB/s eta 0:00:00


In [5]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

## TensorFlow custom training loop using tf.GradientTape

In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Fill in normalization factor
x_test = x_test / 255.0   # Fill in normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Prepare datasets
batch_size = 32         # Fill same batch size as in first TF example
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # Fill size
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Fill number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Fill number of neurons and activation
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")


Epoch 1/5
Step 0, Loss: 2.3613, Accuracy: 0.1875
Step 100, Loss: 0.7241, Accuracy: 0.7305
Step 200, Loss: 0.3428, Accuracy: 0.8016
Step 300, Loss: 0.2079, Accuracy: 0.8328
Step 400, Loss: 0.1654, Accuracy: 0.8509
Step 500, Loss: 0.1209, Accuracy: 0.8630
Step 600, Loss: 0.1894, Accuracy: 0.8732
Step 700, Loss: 0.3606, Accuracy: 0.8793
Step 800, Loss: 0.7126, Accuracy: 0.8851
Step 900, Loss: 0.3033, Accuracy: 0.8893
Step 1000, Loss: 0.3661, Accuracy: 0.8939
Step 1100, Loss: 0.4547, Accuracy: 0.8972
Step 1200, Loss: 0.0738, Accuracy: 0.8993
Step 1300, Loss: 0.0947, Accuracy: 0.9018
Step 1400, Loss: 0.2581, Accuracy: 0.9040
Step 1500, Loss: 0.0676, Accuracy: 0.9067
Step 1600, Loss: 0.3314, Accuracy: 0.9092
Step 1700, Loss: 0.1227, Accuracy: 0.9115
Step 1800, Loss: 0.0445, Accuracy: 0.9137
Training Accuracy for epoch 1: 0.9149

Epoch 2/5
Step 0, Loss: 0.0743, Accuracy: 1.0000
Step 100, Loss: 0.1383, Accuracy: 0.9567
Step 200, Loss: 0.1641, Accuracy: 0.9527
Step 300, Loss: 0.2350, Accuracy:

## Performance Otimization with Graph Execution using @tf.function

In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Fill in normalization factor
x_test = x_test / 255.0   # Fill in normalization factor
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # Fill size
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Fill number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Fill number of neurons and activation
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  # compile the function into a graph
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")


Epoch 1/5
Step 0, Loss: 2.4223, Accuracy: 0.0938
Step 100, Loss: 0.7230, Accuracy: 0.6962
Step 200, Loss: 0.4437, Accuracy: 0.7833
Step 300, Loss: 0.3501, Accuracy: 0.8219
Step 400, Loss: 0.4388, Accuracy: 0.8431
Step 500, Loss: 0.3511, Accuracy: 0.8579
Step 600, Loss: 0.2120, Accuracy: 0.8687
Step 700, Loss: 0.1400, Accuracy: 0.8766
Step 800, Loss: 0.1338, Accuracy: 0.8833
Step 900, Loss: 0.2405, Accuracy: 0.8882
Step 1000, Loss: 0.3075, Accuracy: 0.8922
Step 1100, Loss: 0.1538, Accuracy: 0.8957
Step 1200, Loss: 0.1074, Accuracy: 0.8982
Step 1300, Loss: 0.0861, Accuracy: 0.9004
Step 1400, Loss: 0.0884, Accuracy: 0.9028
Step 1500, Loss: 0.0677, Accuracy: 0.9057
Step 1600, Loss: 0.0667, Accuracy: 0.9080
Step 1700, Loss: 0.1196, Accuracy: 0.9106
Step 1800, Loss: 0.2693, Accuracy: 0.9127
Training Accuracy for epoch 1: 0.9145

Epoch 2/5
Step 0, Loss: 0.1545, Accuracy: 0.9688
Step 100, Loss: 0.0335, Accuracy: 0.9508
Step 200, Loss: 0.0467, Accuracy: 0.9515
Step 300, Loss: 0.0555, Accuracy: